In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [27]:
path = '../DataSets/'
data = 'application_data.csv'

### ***Lectura de data cruda***

In [28]:
df = pd.read_csv(path + data)
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### ***Eliminación de columnas innecesarias***

In [29]:
# Encuentro el indice de la columna EXT_SOURCE_1
ext_source_1_index = df.columns.get_loc('EXT_SOURCE_1')
# Elimino columnas desde EXT_SOURCE_1 en adelante
df = df.iloc[:, :ext_source_1_index].drop(columns=['OWN_CAR_AGE'])

In [30]:
# Tamaño de df reducido
df.shape

(307511, 40)

In [31]:
# Columnas
print(df.columns)

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'ORGANIZATION_TYPE'],
      dtype='object')


### ***Valores Nulos***

In [32]:
# Valores nulos por columna
# df[['AMT_GOODS_PRICE','NAME_TYPE_SUITE','OCCUPATION_TYPE']].isnull().sum()

# La columna AMT_GOODS_PRICE    278     0.09 %
# La columna NAME_TYPE_SUITE    1292    0.42 %
# La columna OCCUPATION_TYPE    96391   31.35 %

print("Valores nulos por columna (luego de eliminar las que no van):")

print("""
Columna                 Nulos     Porcentaje      Descripción
AMT_GOODS_PRICE         278       0.09  %         For consumer loans it is the price of the goods for which the loan is given
NAME_TYPE_SUITE         1292      0.42  %         Who was accompanying client when he was applying for the loan
OCCUPATION_TYPE         96391     31.35 %         What kind of occupation does the client have
AMT_ANNUITY             12
CNT_FAM_MEMBERS         2
""")

print(f'\nTamaño del Dataset resultante: {df.shape}')

Valores nulos por columna (luego de eliminar las que no van):

Columna                 Nulos     Porcentaje      Descripción
AMT_GOODS_PRICE         278       0.09  %         For consumer loans it is the price of the goods for which the loan is given
NAME_TYPE_SUITE         1292      0.42  %         Who was accompanying client when he was applying for the loan
OCCUPATION_TYPE         96391     31.35 %         What kind of occupation does the client have
AMT_ANNUITY             12
CNT_FAM_MEMBERS         2


Tamaño del Dataset resultante: (307511, 40)


In [33]:
# Revisamos la columna CNT_FAM_MEMBERS, notamos que falta el número 0
df['CNT_FAM_MEMBERS'].value_counts()

CNT_FAM_MEMBERS
2.0     158357
1.0      67847
3.0      52601
4.0      24697
5.0       3478
6.0        408
7.0         81
8.0         20
9.0          6
10.0         3
14.0         2
12.0         2
20.0         2
16.0         2
13.0         1
15.0         1
11.0         1
Name: count, dtype: int64

In [34]:
# Podemos notar que los valores nulos de CNT_FAM_MEMBERS cuentan con valores 0 en CNT_CHILDREN.
# Por lo que la solución seria inputar los nulos con 0.
num_nulls = df['CNT_FAM_MEMBERS'].isnull().sum()
null_rows = df[df['CNT_FAM_MEMBERS'].isnull()]
null_rows

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE
41982,148605,0,Revolving loans,M,N,Y,0,450000.0,675000.0,33750.0,...,2,THURSDAY,15,0,1,1,0,1,1,Insurance
187348,317181,0,Revolving loans,F,N,Y,0,202500.0,585000.0,29250.0,...,2,FRIDAY,14,0,0,0,0,0,0,Business Entity Type 2


### ***Imputación de datos nulos REVISAR***

In [35]:
# Traigo la media de la columna AMT_GOODS_PRICE para realizar imputación de datos
amt_goods_price_mean = df['AMT_GOODS_PRICE'].mean()
amt_annuity_mean = df['AMT_ANNUITY'].mean()

In [36]:
value_input = {
    'OCCUPATION_TYPE': 'Other',
    'NAME_TYPE_SUITE': 'Other',
    'AMT_ANNUITY': amt_annuity_mean,
    'AMT_GOODS_PRICE': amt_goods_price_mean,
    'CNT_FAM_MEMBERS': 0
}
df = df.fillna(value=value_input)

### ***Reemplazo de valores en columna NAME_TYPE_SUITE***
Noto que hay una categoria 'Other_A' y 'Other_B'. Y una posible solucion sería unificar las categorias a 'Other'

In [37]:
# NAME_TYPE_SUITE
df['NAME_TYPE_SUITE'].value_counts()

NAME_TYPE_SUITE
Unaccompanied      248526
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other                1292
Other_A               866
Group of people       271
Name: count, dtype: int64

In [38]:
# Reemplazo los valores Other_A y Other_B por Other
df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].replace({'Other_A': 'Other', 'Other_B': 'Other'})

### ***Reemplazo de valores en columna ORGANIZATION_TYPE***
Noto que las categorias se dividen en subcategorias por tipo. Una posible solución es unificar las categorías.

In [39]:
df['ORGANIZATION_TYPE'].value_counts()

ORGANIZATION_TYPE
Business Entity Type 3    67992
XNA                       55374
Self-employed             38412
Other                     16683
Medicine                  11193
Business Entity Type 2    10553
Government                10404
School                     8893
Trade: type 7              7831
Kindergarten               6880
Construction               6721
Business Entity Type 1     5984
Transport: type 4          5398
Trade: type 3              3492
Industry: type 9           3368
Industry: type 3           3278
Security                   3247
Housing                    2958
Industry: type 11          2704
Military                   2634
Bank                       2507
Agriculture                2454
Police                     2341
Transport: type 2          2204
Postal                     2157
Security Ministries        1974
Trade: type 2              1900
Restaurant                 1811
Services                   1575
University                 1327
Industry: type 7      

In [40]:
replace_dict = {
    'Business Entity Type 1': 'Business Entity',
    'Business Entity Type 2': 'Business Entity',
    'Business Entity Type 3': 'Business Entity',
    'Trade: type 1': 'Trade',
    'Trade: type 2': 'Trade',
    'Trade: type 3': 'Trade',
    'Trade: type 4': 'Trade',
    'Trade: type 5': 'Trade',
    'Trade: type 6': 'Trade',
    'Trade: type 7': 'Trade',
    'Transport: type 1': 'Transport',
    'Transport: type 2': 'Transport',
    'Transport: type 3': 'Transport',
    'Transport: type 4': 'Transport',
    'Industry: type 1': 'Industry',
    'Industry: type 2': 'Industry',
    'Industry: type 3': 'Industry',
    'Industry: type 4': 'Industry',
    'Industry: type 5': 'Industry',
    'Industry: type 6': 'Industry',
    'Industry: type 7': 'Industry',
    'Industry: type 8': 'Industry',
    'Industry: type 9': 'Industry',
    'Industry: type 10': 'Industry',
    'Industry: type 11': 'Industry',
    'Industry: type 12': 'Industry',
    'Industry: type 13': 'Industry'
}

# Reemplazar los valores en la columna ORGANIZATION_TYPE
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace(replace_dict)

In [41]:
df['ORGANIZATION_TYPE'].value_counts()

ORGANIZATION_TYPE
Business Entity        84529
XNA                    55374
Self-employed          38412
Other                  16683
Trade                  14315
Industry               14311
Medicine               11193
Government             10404
Transport               8990
School                  8893
Kindergarten            6880
Construction            6721
Security                3247
Housing                 2958
Military                2634
Bank                    2507
Agriculture             2454
Police                  2341
Postal                  2157
Security Ministries     1974
Restaurant              1811
Services                1575
University              1327
Hotel                    966
Electricity              950
Insurance                597
Telecom                  577
Emergency                560
Advertising              429
Realtor                  396
Culture                  379
Mobile                   317
Legal Services           305
Cleaning                 

### ***Tipos de datos***

In [42]:
# Tipo de datos ORIGEN
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int64  
 1   TARGET                       307511 non-null  int64  
 2   NAME_CONTRACT_TYPE           307511 non-null  object 
 3   CODE_GENDER                  307511 non-null  object 
 4   FLAG_OWN_CAR                 307511 non-null  object 
 5   FLAG_OWN_REALTY              307511 non-null  object 
 6   CNT_CHILDREN                 307511 non-null  int64  
 7   AMT_INCOME_TOTAL             307511 non-null  float64
 8   AMT_CREDIT                   307511 non-null  float64
 9   AMT_ANNUITY                  307511 non-null  float64
 10  AMT_GOODS_PRICE              307511 non-null  float64
 11  NAME_TYPE_SUITE              307511 non-null  object 
 12  NAME_INCOME_TYPE             307511 non-null  object 
 13 

In [43]:
new_type_cols = {
    'SK_ID_CURR': 'int64',
    'TARGET': 'int64',
    'NAME_CONTRACT_TYPE': 'category',
    'CODE_GENDER': 'category',
    'FLAG_OWN_CAR': 'category',
    'FLAG_OWN_REALTY': 'category',
    'CNT_CHILDREN': 'int64',
    'AMT_INCOME_TOTAL': 'float64',
    'AMT_CREDIT': 'float64',
    'AMT_ANNUITY': 'float64',
    'AMT_GOODS_PRICE': 'float64',
    'NAME_TYPE_SUITE': 'category',
    'NAME_INCOME_TYPE': 'category',
    'NAME_EDUCATION_TYPE': 'category',
    'NAME_FAMILY_STATUS': 'category',
    'NAME_HOUSING_TYPE': 'category',
    'REGION_POPULATION_RELATIVE': 'float64',
    'DAYS_BIRTH': 'int64',
    'DAYS_EMPLOYED': 'int64',
    'DAYS_REGISTRATION': 'float64',
    'DAYS_ID_PUBLISH': 'int64',
    'FLAG_MOBIL': 'int64',
    'FLAG_EMP_PHONE': 'int64',
    'FLAG_WORK_PHONE': 'int64',
    'FLAG_CONT_MOBILE': 'int64',
    'FLAG_PHONE': 'int64',
    'FLAG_EMAIL': 'int64',
    'OCCUPATION_TYPE': 'category',
    'CNT_FAM_MEMBERS': 'int64',
    'REGION_RATING_CLIENT': 'int64',
    'REGION_RATING_CLIENT_W_CITY': 'int64',
    'WEEKDAY_APPR_PROCESS_START': 'category',
    'HOUR_APPR_PROCESS_START': 'int64',
    'REG_REGION_NOT_LIVE_REGION': 'int64',
    'REG_REGION_NOT_WORK_REGION': 'int64',
    'LIVE_REGION_NOT_WORK_REGION': 'int64',
    'REG_CITY_NOT_LIVE_CITY': 'int64',
    'REG_CITY_NOT_WORK_CITY': 'int64',
    'LIVE_CITY_NOT_WORK_CITY': 'int64',
    'ORGANIZATION_TYPE': 'category'
}
df = df.astype(new_type_cols)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   SK_ID_CURR                   307511 non-null  int64   
 1   TARGET                       307511 non-null  int64   
 2   NAME_CONTRACT_TYPE           307511 non-null  category
 3   CODE_GENDER                  307511 non-null  category
 4   FLAG_OWN_CAR                 307511 non-null  category
 5   FLAG_OWN_REALTY              307511 non-null  category
 6   CNT_CHILDREN                 307511 non-null  int64   
 7   AMT_INCOME_TOTAL             307511 non-null  float64 
 8   AMT_CREDIT                   307511 non-null  float64 
 9   AMT_ANNUITY                  307511 non-null  float64 
 10  AMT_GOODS_PRICE              307511 non-null  float64 
 11  NAME_TYPE_SUITE              307511 non-null  category
 12  NAME_INCOME_TYPE             307511 non-null

### ***Nombre de Columnas***

In [44]:
renamed_cols = {
    'SK_ID_CURR': 'ID_CREDITO',
    'TARGET': 'OBJETIVO',
    'NAME_CONTRACT_TYPE': 'TIPO_PAGO',
    'CODE_GENDER': 'GENERO',
    'FLAG_OWN_CAR': 'PROP_AUTO',
    'FLAG_OWN_REALTY': 'PROP_INMUEBLE',
    'CNT_CHILDREN': 'CANTIDAD_HIJOS',
    'AMT_INCOME_TOTAL': 'INGRESO',
    'AMT_CREDIT': 'MONTO_CREDITO',
    'AMT_ANNUITY': 'ANUALIDAD_PRESTAMO',
    'AMT_GOODS_PRICE': 'PRECIO_BIENES',
    'NAME_TYPE_SUITE': 'TIPO_ACOMPAÑANTE',
    'NAME_INCOME_TYPE': 'TIPO_INGRESO',
    'NAME_EDUCATION_TYPE': 'NIVEL_ESTUDIO',
    'NAME_FAMILY_STATUS': 'ESTADO_CIVIL',
    'NAME_HOUSING_TYPE': 'TIPO_VIVIENDA',
    'REGION_POPULATION_RELATIVE': 'POBLACION_RELATIVA_REGION',
    'DAYS_BIRTH': 'EDAD_DIAS',
    'DAYS_EMPLOYED': 'DIAS_EMPLEADO',
    'DAYS_REGISTRATION': 'DIAS_MODIF_REGISTRO',
    'DAYS_ID_PUBLISH': 'DIAS_MODIF_DOCUMENTO',
    'FLAG_MOBIL': 'TIENE_CELULAR',
    'FLAG_EMP_PHONE': 'TELEFONO_EMPLEO',
    'FLAG_WORK_PHONE': 'TELEFONO_TRABAJO',
    'FLAG_CONT_MOBILE': 'ATIENDE_CELULAR',
    'FLAG_PHONE': 'TELEFONO_CASA',
    'FLAG_EMAIL': 'EMAIL',
    'OCCUPATION_TYPE': 'PROFESION',
    'CNT_FAM_MEMBERS': 'CANTIDAD_MIEMBROS_FAMILIA',
    'REGION_RATING_CLIENT': 'CALIFICACION_REGION_CLIENTE',
    'REGION_RATING_CLIENT_W_CITY': 'CALIFICACION_REGION_CLIENTE_CIUDAD',
    'WEEKDAY_APPR_PROCESS_START': 'DIA_SEMANA_SOLICITUD',
    'HOUR_APPR_PROCESS_START': 'HORA_SOLICITUD',
    'REG_REGION_NOT_LIVE_REGION': 'DIRECCION_PERMANENTE_NO_DIRECCION_CONTACTO',
    'REG_REGION_NOT_WORK_REGION': 'DIRECCION_PERMANENTE_NO_DIRECCION_TRABAJO',
    'LIVE_REGION_NOT_WORK_REGION': 'DIRECCION_CONTACTO_NO_DIRECCION_TRABAJO',
    'REG_CITY_NOT_LIVE_CITY': 'NO_VIVE_CIUDAD_REGISTRADA',
    'REG_CITY_NOT_WORK_CITY': 'NO_TRABAJA_CIUDAD_REGISTRADA',
    'LIVE_CITY_NOT_WORK_CITY': 'NO_VIVE_CIUDAD_DE_TRABAJO',
    'ORGANIZATION_TYPE': 'TIPO_ORGANIZACION'
}
df = df.rename(columns=renamed_cols)

In [45]:
df.head()

,ID_CREDITO,OBJETIVO,TIPO_PAGO,GENERO,PROP_AUTO,PROP_INMUEBLE,CANTIDAD_HIJOS,INGRESO,MONTO_CREDITO,ANUALIDAD_PRESTAMO,...,CALIFICACION_REGION_CLIENTE_CIUDAD,DIA_SEMANA_SOLICITUD,HORA_SOLICITUD,DIRECCION_PERMANENTE_NO_DIRECCION_CONTACTO,DIRECCION_PERMANENTE_NO_DIRECCION_TRABAJO,DIRECCION_CONTACTO_NO_DIRECCION_TRABAJO,NO_VIVE_CIUDAD_REGISTRADA,NO_TRABAJA_CIUDAD_REGISTRADA,NO_VIVE_CIUDAD_DE_TRABAJO,TIPO_ORGANIZACION
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1,MONDAY,11,0,0,0,0,0,0,School
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,2,MONDAY,9,0,0,0,0,0,0,Government
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,2,THURSDAY,11,0,0,0,0,1,1,Religion


In [47]:
# SCRIPT COMPLETO ETL

## SCRIPT ETL
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

path = '/content/drive/MyDrive/ProyectoNoCountry/data2/'
data = 'application_data.csv'

df = pd.read_csv(path + data)

# Elimino columnas desde EXT_SOURCE_1 en adelante
ext_source_1_index = df.columns.get_loc('EXT_SOURCE_1')
df = df.iloc[:, :ext_source_1_index].drop(columns=['OWN_CAR_AGE'])

# Imputación de datos nulos
amt_goods_price_mean = df['AMT_GOODS_PRICE'].mean()
amt_annuity_mean = df['AMT_ANNUITY'].mean()

value_input = {
    'OCCUPATION_TYPE': 'Other',
    'NAME_TYPE_SUITE': 'Other',
    'AMT_ANNUITY': amt_annuity_mean,
    'AMT_GOODS_PRICE': amt_goods_price_mean,
    'CNT_FAM_MEMBERS': 0
    }
df = df.fillna(value=value_input)

# Reemplazo los valores Other_A y Other_B por Other
df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].replace({'Other_A': 'Other', 'Other_B': 'Other'})

# cambio tipos de datos
new_type_cols = {
    'SK_ID_CURR': 'int64',
    'TARGET': 'int64',
    'NAME_CONTRACT_TYPE': 'category',
    'CODE_GENDER': 'category',
    'FLAG_OWN_CAR': 'category',
    'FLAG_OWN_REALTY': 'category',
    'CNT_CHILDREN': 'int64',
    'AMT_INCOME_TOTAL': 'float64',
    'AMT_CREDIT': 'float64',
    'AMT_ANNUITY': 'float64',
    'AMT_GOODS_PRICE': 'float64',
    'NAME_TYPE_SUITE': 'category',
    'NAME_INCOME_TYPE': 'category',
    'NAME_EDUCATION_TYPE': 'category',
    'NAME_FAMILY_STATUS': 'category',
    'NAME_HOUSING_TYPE': 'category',
    'REGION_POPULATION_RELATIVE': 'float64',
    'DAYS_BIRTH': 'int64',
    'DAYS_EMPLOYED': 'int64',
    'DAYS_REGISTRATION': 'float64',
    'DAYS_ID_PUBLISH': 'int64',
    'FLAG_MOBIL': 'int64',
    'FLAG_EMP_PHONE': 'int64',
    'FLAG_WORK_PHONE': 'int64',
    'FLAG_CONT_MOBILE': 'int64',
    'FLAG_PHONE': 'int64',
    'FLAG_EMAIL': 'int64',
    'OCCUPATION_TYPE': 'category',
    'CNT_FAM_MEMBERS': 'int64',
    'REGION_RATING_CLIENT': 'int64',
    'REGION_RATING_CLIENT_W_CITY': 'int64',
    'WEEKDAY_APPR_PROCESS_START': 'category',
    'HOUR_APPR_PROCESS_START': 'int64',
    'REG_REGION_NOT_LIVE_REGION': 'int64',
    'REG_REGION_NOT_WORK_REGION': 'int64',
    'LIVE_REGION_NOT_WORK_REGION': 'int64',
    'REG_CITY_NOT_LIVE_CITY': 'int64',
    'REG_CITY_NOT_WORK_CITY': 'int64',
    'LIVE_CITY_NOT_WORK_CITY': 'int64',
    'ORGANIZATION_TYPE': 'category'
    }
df = df.astype(new_type_cols)

# cambio nombre de columnas
renamed_cols = {
    'SK_ID_CURR': 'ID_CREDITO',
    'TARGET': 'OBJETIVO',
    'NAME_CONTRACT_TYPE': 'TIPO_PAGO',
    'CODE_GENDER': 'GENERO',
    'FLAG_OWN_CAR': 'PROP_AUTO',
    'FLAG_OWN_REALTY': 'PROP_INMUEBLE',
    'CNT_CHILDREN': 'CANTIDAD_HIJOS',
    'AMT_INCOME_TOTAL': 'INGRESO',
    'AMT_CREDIT': 'MONTO_CREDITO',
    'AMT_ANNUITY': 'ANUALIDAD_PRESTAMO',
    'AMT_GOODS_PRICE': 'PRECIO_BIENES',
    'NAME_TYPE_SUITE': 'TIPO_ACOMPAÑANTE',
    'NAME_INCOME_TYPE': 'TIPO_INGRESO',
    'NAME_EDUCATION_TYPE': 'NIVEL_ESTUDIO',
    'NAME_FAMILY_STATUS': 'ESTADO_CIVIL',
    'NAME_HOUSING_TYPE': 'TIPO_VIVIENDA',
    'REGION_POPULATION_RELATIVE': 'POBLACION_RELATIVA_REGION',
    'DAYS_BIRTH': 'EDAD_DIAS',
    'DAYS_EMPLOYED': 'DIAS_EMPLEADO',
    'DAYS_REGISTRATION': 'DIAS_MODIF_REGISTRO',
    'DAYS_ID_PUBLISH': 'DIAS_MODIF_DOCUMENTO',
    'FLAG_MOBIL': 'TIENE_CELULAR',
    'FLAG_EMP_PHONE': 'TELEFONO_EMPLEO',
    'FLAG_WORK_PHONE': 'TELEFONO_TRABAJO',
    'FLAG_CONT_MOBILE': 'ATIENDE_CELULAR',
    'FLAG_PHONE': 'TELEFONO_CASA',
    'FLAG_EMAIL': 'EMAIL',
    'OCCUPATION_TYPE': 'PROFESION',
    'CNT_FAM_MEMBERS': 'CANTIDAD_MIEMBROS_FAMILIA',
    'REGION_RATING_CLIENT': 'CALIFICACION_REGION_CLIENTE',
    'REGION_RATING_CLIENT_W_CITY': 'CALIFICACION_REGION_CLIENTE_CIUDAD',
    'WEEKDAY_APPR_PROCESS_START': 'DIA_SEMANA_SOLICITUD',
    'HOUR_APPR_PROCESS_START': 'HORA_SOLICITUD',
    'REG_REGION_NOT_LIVE_REGION': 'DIRECCION_PERMANENTE_NO_DIRECCION_CONTACTO',
    'REG_REGION_NOT_WORK_REGION': 'DIRECCION_PERMANENTE_NO_DIRECCION_TRABAJO',
    'LIVE_REGION_NOT_WORK_REGION': 'DIRECCION_CONTACTO_NO_DIRECCION_TRABAJO',
    'REG_CITY_NOT_LIVE_CITY': 'NO_VIVE_CIUDAD_REGISTRADA',
    'REG_CITY_NOT_WORK_CITY': 'NO_TRABAJA_CIUDAD_REGISTRADA',
    'LIVE_CITY_NOT_WORK_CITY': 'NO_VIVE_CIUDAD_DE_TRABAJO',
    'ORGANIZATION_TYPE': 'TIPO_ORGANIZACION'
    }
df = df.rename(columns=renamed_cols)

data = data.replace('.csv', '.parquet')
#path_data_transform = f'{path}{data}'

# conversion a parquet
table = pa.Table.from_pandas(df)
pq.write_table(table, path + data)

### ***Verifico leyendo los datos procesados***

#### De la siguiente manera se pueden leer los datos en formato parquet

In [5]:
data_procesada = pd.read_parquet('../DataSets/application_data_procesada.parquet')
data_procesada

,ID_CREDITO,OBJETIVO,TIPO_PAGO,GENERO,PROP_AUTO,PROP_INMUEBLE,CANTIDAD_HIJOS,INGRESO,MONTO_CREDITO,ANUALIDAD_PRESTAMO,...,CALIFICACION_REGION_CLIENTE_CIUDAD,DIA_SEMANA_SOLICITUD,HORA_SOLICITUD,DIRECCION_PERMANENTE_NO_DIRECCION_CONTACTO,DIRECCION_PERMANENTE_NO_DIRECCION_TRABAJO,DIRECCION_CONTACTO_NO_DIRECCION_TRABAJO,NO_VIVE_CIUDAD_REGISTRADA,NO_TRABAJA_CIUDAD_REGISTRADA,NO_VIVE_CIUDAD_DE_TRABAJO,TIPO_ORGANIZACION
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1,MONDAY,11,0,0,0,0,0,0,School
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,2,MONDAY,9,0,0,0,0,0,0,Government
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,2,THURSDAY,11,0,0,0,0,1,1,Religion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,1,THURSDAY,15,0,0,0,0,0,0,Services
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,2,MONDAY,8,0,0,0,0,0,0,XNA
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,3,THURSDAY,9,0,0,0,0,1,1,School
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,2,WEDNESDAY,9,0,0,0,1,1,0,Business Entity Type 1


In [6]:
data_procesada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 40 columns):
 #   Column                                      Non-Null Count   Dtype   
---  ------                                      --------------   -----   
 0   ID_CREDITO                                  307511 non-null  int64   
 1   OBJETIVO                                    307511 non-null  int64   
 2   TIPO_PAGO                                   307511 non-null  category
 3   GENERO                                      307511 non-null  category
 4   PROP_AUTO                                   307511 non-null  category
 5   PROP_INMUEBLE                               307511 non-null  category
 6   CANTIDAD_HIJOS                              307511 non-null  int64   
 7   INGRESO                                     307511 non-null  float64 
 8   MONTO_CREDITO                               307511 non-null  float64 
 9   ANUALIDAD_PRESTAMO                          307511 non-null

# Dataframe con detalle de columnas

In [ ]:
data = 'data_description_spanish.csv'

df_description = pd.read_csv(path + data)
df_description

,Row,Descripción
0,SK_ID_CURR,ID del préstamo en nuestra muestra
1,TARGET,Variable objetivo (1 - cliente con dificultade...
2,NAME_CONTRACT_TYPE,Identificación si el préstamo es en efectivo o...
3,CODE_GENDER,Género del cliente
4,FLAG_OWN_CAR,Bandera si el cliente posee un coche
5,FLAG_OWN_REALTY,Bandera si el cliente posee una casa o apartam...
6,CNT_CHILDREN,Número de hijos que tiene el cliente
7,AMT_INCOME_TOTAL,Ingreso del cliente
8,AMT_CREDIT,Cantidad de crédito del préstamo
9,AMT_ANNUITY,Anualidad del préstamo
